# Student Alcohol Consumption Analysiss

## Introduction
This Jupyter Notebook aims to analyze the factors influencing alcohol consumption among secondary school students. By exploring the dataset, we will gain insights into how various social, educational, and personal factors affect students' alcohol consumption patterns and academic performance.

## Table of Contents
1. [Package Installation](#package-installation)
2. [Library Imports](#library-imports)
3. [Data Loading](#data-loading)
4. [Initial Data Exploration](#initial-data-exploration)
5. [Data Visualization](#data-visualization)
6. [Correlation Analysis](#correlation-analysis)
7. [Data Preparation](#data-preparation)
8. [Modeling](#modeling)
9. [Model Evaluation](#model-evaluation)
10. [Conclusion and Next Steps](#conclusion-and-next-steps)

## Package Installation <a name="package-installation"></a>
In this cell, we will install the necessary Python packages required for our data analysis project. This step ensures that all the libraries needed for data manipulation, visualization, and machine learning are available in our environment. 

We will use the `pip` command to install the following libraries:

- **NumPy**: A library for numerical computations and handling arrays.
- **Pandas**: A powerful data manipulation and analysis library, particularly useful for working with structured data.
- **Matplotlib**: A plotting library for creating static, animated, and interactive visualizations in Python.
- **Seaborn**: A statistical data visualization library based on Matplotlib that provides a high-level interface for drawing attractive graphics.
- **Scikit-learn**: A machine learning library that provides simple and efficient tools for data mining and data analysis.

In [ ]:
# List of required libraries
required_libraries = [
    'numpy',
    'pandas',
    'matplotlib',
    'seaborn',
    'scikit-learn'
]

def install(package):
    """Install the package using pip in a Jupyter Notebook."""
    print(f"Installing {package}...")
    # Use the Jupyter magic command for installation
    get_ipython().system(f'pip install {package}')

def check_libraries(libraries):
    """Check if the libraries are installed and install them if necessary."""
    missing_libraries = []

    for library in libraries:
        try:
            __import__(library)
        except ImportError:
            missing_libraries.append(library)
        except Exception as e:
            # Captura outros erros que podem ocorrer durante a importação
            print(f"Error importing {library}: {e}")
            missing_libraries.append(library)

    if missing_libraries:
        print(f"The following libraries are missing: {', '.join(missing_libraries)}")
        print("Starting installation...")

        installation_success = True  # Flag to track installation success

        for library in missing_libraries:
            try:
                install(library)
                print(f"{library} installed successfully.")
            except Exception as e:
                print(f"Failed to install {library}: {e}")
                installation_success = False  # Mark as failed if there was an error

        # Check again if the libraries were installed
        for library in missing_libraries:
            try:
                __import__(library)
            except ImportError:
                print(f"Error: {library} was not installed correctly.")
                installation_success = False  # Mark as failed if still missing

        # Final message based on installation success
        if installation_success:
            print("All libraries were installed successfully.")
        else:
            print("Some libraries were not installed correctly.")
    else:
        print("All libraries are already installed.")

if __name__ == "__main__":
    check_libraries(required_libraries)

## Library Imports <a name="library-imports"></a>
In this cell, we will import all the necessary libraries that we will use throughout the analysis. This includes libraries for data manipulation, visualization, and machine learning.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## Data Loading <a name="data-loading"></a>
Here, we will load the dataset containing information about students' alcohol consumption and related factors. We will examine the structure of the data and check for any initial issues such as missing values.

In [3]:
# Paths to the CSV files
path_data_math = '../data/student-mat.csv'
path_data_port = '../data/student-por.csv'

# Reading the CSV files
data_math = pd.read_csv(path_data_math)
data_port = pd.read_csv(path_data_port)

In [ ]:
# Display the first few rows of each dataset to verify the reading
print("Mathematics Data:")
print(data_math.head())
print("\nPortuguese Data:")
print(data_port.head())

In [5]:
# Concatenating the two DataFrames
combined_data = pd.merge(data_math, data_port, on=["school", "sex", "age", "address", "famsize", 
                             "Pstatus", "Medu", "Fedu", "Mjob", "Fjob", 
                             "reason", "nursery", "internet"],
                             suffixes=('_math', '_port'))

# Using shape to check the dimensions of the combined DataFrame
print("\nShape of Combined Data:", combined_data.shape)


Shape of Combined Data: (382, 53)


## Initial Data Exploration <a name="initial-data-exploration"></a>
In this section, we will perform an exploratory data analysis (EDA) to understand the dataset better. We will look at the data types, summary statistics, and any missing values, which will help us determine the next steps for cleaning and preparing the data.

In [ ]:
# 1. Check for missing values
missing_values = combined_data.isnull().sum()
print("Missing Values in Each Column:\n", missing_values)

In [ ]:
# 2. Check data types
data_types = combined_data.dtypes
print("\nData Types of Each Column:\n", data_types)

In [ ]:
# 3. Correcting Data Types for Categorical Variables

# List the columns to convert to categorical
columns_to_convert = [
    'school', 'sex', 'address', 'famsize', 'Pstatus',
    'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian_math',
    'guardian_port', 'schoolsup_math', 'schoolsup_port', 'famsup_math',
    'famsup_port', 'paid_math', 'paid_port', 'activities_math', 'activities_port', 
    'nursery', 'higher_math', 'higher_port', 'internet', 'romantic_math',
    'romantic_port', 'traveltime_math', 'traveltime_port','studytime_math',
    'studytime_port', 'failures_math', 'failures_port', 'famrel_math', 'famrel_port',
    'freetime_math', 'freetime_port', 'goout_math', 'goout_port', 'Dalc_math', 'Dalc_port',
    'Walc_math', 'Walc_port', 'health_math', 'health_port'
]

# Convert the specified columns to 'category' type
combined_data[columns_to_convert] = combined_data[columns_to_convert].astype('category')

# Check the data types after conversion
print("Updated Data Types of Each Column:")
print(combined_data.dtypes)

In [ ]:
# 4. Summary statistics for numerical columns
summary_statistics = combined_data.describe()
print("\nSummary Statistics:\n", summary_statistics)

In [ ]:
# 5. Check unique values in categorical columns
for column in combined_data.select_dtypes(include=['category']).columns:
    print(f"\nUnique Values in '{column}':\n", combined_data[column].value_counts())

In [ ]:
# 6. Visualize distributions for numerical columns using histograms
combined_data.hist(bins=30, figsize=(15, 10))
plt.tight_layout()
plt.show()

In [ ]:
# 7. Create boxplots to check for outliers in numerical columns
for column in combined_data.select_dtypes(exclude=['category']).columns:
    plt.figure(figsize=(10, 5))
    sns.boxplot(x=combined_data[column])
    plt.title(f'Boxplot of {column}')
    plt.show()

After analyzing the outliers in the dataset, we have decided to retain them as they represent valid data points that are relevant for subsequent evaluations.

## Data Visualization <a name="data-visualization"></a>
Data visualization is key to understanding trends and patterns in our data. In this cell, we will create various plots to visualize the relationships between different variables, including alcohol consumption and academic performance.

In [8]:
# Set style for the plots
sns.set(style="whitegrid")

In [25]:
# Function to create volin plots
def plot_violin(alcohol_col, grades_cols, subject, hue_col=None):
    for grade_col in grades_cols:
        plt.figure(figsize=(12, 6))

        # Gráfico de violino
        if hue_col:
            sns.violinplot(data=df, x=alcohol_col, y=grade_col, hue=hue_col, palette='deep', split=True)
        else:
            sns.violinplot(data=df, x=alcohol_col, y=grade_col, palette='deep')

        plt.title(f'Violin Plot of {grade_col} by {alcohol_col} in {subject}')
        plt.xlabel(alcohol_col)
        plt.ylabel(grade_col)
        plt.grid()
        plt.show()

In [27]:
math_grades = ['G1_math', 'G2_math', 'G3_math']
# Chamada da função com uma coluna hue, por exemplo, 'gender'
plot_violin('Dalc_math', math_grades, 'Math', hue_col='gender')
plot_violin('Walc_math', math_grades, 'Math', hue_col='gender')

NameError: name 'df' is not defined

<Figure size 1200x600 with 0 Axes>

In [ ]:
# Variables for Portuguese
portuguese_grades = ['G1_port', 'G2_port', 'G3_port']
plot_violin('Dalc_port', portuguese_grades, 'Portuguese')
plot_violin('Walc_port', portuguese_grades, 'Portuguese')

## Correlation Analysis <a name="correlation-analysis"></a>
We will analyze the correlations between numerical features in the dataset. This will help us identify which factors are most strongly related to alcohol consumption and academic performance.


## Data Preparation <a name="data-preparation"></a>
Before modeling, we need to prepare the data. This includes handling missing values, encoding categorical variables, and splitting the dataset into training and testing sets.

## Modeling <a name="modeling"></a>
In this section, we will build machine learning models to predict students' academic performance based on their alcohol consumption and other features. We will choose appropriate algorithms and fit them to our training data.

## Model Evaluation <a name="model-evaluation"></a>
After building our models, we will evaluate their performance using various metrics. We will compare the predictions against the actual outcomes to determine how well our models are performing.

## Conclusion and Next Steps <a name="conclusion-and-next-steps"></a>
In the final section, we will summarize our findings from the analysis and discuss potential next steps. This may include further analysis, model improvements, or recommendations for educators based on the insights gained from the data.